## Pricing and Order Placement

This sample shows how to retrieve price data for FX instruments and place a number of different types of orders.

> Note: pricing data might not be available for your client for non-FX instruments - this sample uses FxSpot which _should_ be available on any vanilla developer account.

Take care running this notebook with a LIVE client, as it _will_ place orders on your account!

In [1]:
from saxo_apy import SaxoOpenAPIClient

client = SaxoOpenAPIClient()
client.login()

🌐 opening login page in browser - waiting for user to authenticate... 🔑
📞 received callback from Saxo SSO
✅ authorization succeeded - connected to SIM environment with WRITE / TRADE permissions (session ID 1925c0646e024b2580970f98b49ad1ce)


In [2]:
# find Euro FX crosses
search = {
    "Keywords": "EUR",
    "AssetTypes": "FxSpot",
}

result = client.get("/ref/v1/instruments", params=search)["Data"]

# show first 5 results
[(res["Symbol"], res["Identifier"], res["AssetType"]) for res in result][:5]

[('EURUSD', 21, 'FxSpot'),
 ('EURCHF', 14, 'FxSpot'),
 ('EURJPY', 18, 'FxSpot'),
 ('EURGBP', 17, 'FxSpot'),
 ('EURCAD', 13, 'FxSpot')]

In [3]:
# select EURUSD FX cross and retrieve detailed instrument information
eurusd_details = client.get(f"/ref/v1/instruments/details/{result[0]['Identifier']}/{result[0]['AssetType']}")

# show trade status, available accounts, and available order types
print(f"{eurusd_details['TradingStatus']=}")
print(f"{eurusd_details['TradableOn']=}")
print(f"{eurusd_details['SupportedOrderTypes']=}")
print(f"{eurusd_details['TickSizeLimitOrder']=}")
print(f"{eurusd_details['TickSizeStopOrder']=}")

eurusd_details['TradingStatus']='Tradable'
eurusd_details['TradableOn']=['16371609', '16371609_DKK', '16371609_USD']
eurusd_details['SupportedOrderTypes']=['Stop', 'TrailingStop', 'StopLimit', 'Limit', 'Market']
eurusd_details['TickSizeLimitOrder']=1e-05
eurusd_details['TickSizeStopOrder']=1e-05


In [4]:
# retrieve price data
price_params = {
    "Uic": eurusd_details["Uic"],
    "AssetType": eurusd_details["AssetType"],
    "FieldGroups": ["PriceInfo"],
}

eurusd_price = client.get("/trade/v1/infoprices", params=price_params)["Quote"]

print(
    f"Price for {eurusd_details['Symbol']}: {eurusd_price['Mid']} - Market state is: {eurusd_price['MarketState']} - Price delayed by {eurusd_price['DelayedByMinutes']} minutes."
)

Price for EURUSD: 1.07371 - Market state is: Open - Price delayed by 0 minutes.


## Limit Order

In [5]:
# place a limit order to buy 1000 EURUSD 5% under the current MID price
# the selected account is the default account, which is allowed according to the /details response (see above)
# the price level for this order is quantized in order to round it to the closest (allowed) TickSize increment for this order type (see above)

from decimal import Decimal
from secrets import token_urlsafe

order_price = Decimal(eurusd_price["Mid"] * 0.95).quantize(Decimal(str(eurusd_details["TickSizeLimitOrder"])))
print(f"Order price for this order: {order_price}")

client_data = client.get("/port/v1/clients/me")

limit_order = {
    "AccountKey": client_data["DefaultAccountKey"],
    "AssetType": eurusd_details["AssetType"],
    "Uic": eurusd_details["Uic"],
    "BuySell": "Buy",
    "Amount": 1000,
    "OrderType": "Limit",
    "OrderPrice": str(order_price),  # for serialization
    "OrderDuration": {"DurationType": "GoodTillCancel"},
    "ExternalReference": token_urlsafe(16),  # a random identifier for this order
    "ManualOrder": True,
}

placed_order = client.post("/trade/v2/orders", data=limit_order)

print(f"Order placed with id: {placed_order['OrderId']}, external reference: {placed_order['ExternalReference']}")

Order price for this order: 1.02002
Order placed with id: 5010735121, external reference: vDyr_dCPC9sptKGX0wMqoA


In [6]:
# retrieve order status from portfolio
order = client.get(f"/port/v1/orders/{client_data['ClientKey']}/{placed_order['OrderId']}")["Data"][0]

print(f"Order: {order['OrderId']}, status: {order['Status']}")

Order: 5010735121, status: Working


In [7]:
# cancel order
client.delete(f"/trade/v2/orders/{order['OrderId']}?AccountKey={order['AccountKey']}")

# no data is returned because status of delete request is 204 No Content

In [8]:
# confirm order is now removed
client.get(f"/port/v1/orders/{client_data['ClientKey']}/{placed_order['OrderId']}")

{'__count': 0, 'Data': []}

## Stop Order

In [9]:
# similar to the limit order above, we now place a stop order to buy 1000 EURUSD 5% over the current MID price

from decimal import Decimal
from secrets import token_urlsafe

order_price = Decimal(eurusd_price["Mid"] * 1.05).quantize(Decimal(str(eurusd_details["TickSizeStopOrder"])))
print(f"Order price for this order: {order_price}")

client_data = client.get("/port/v1/clients/me")

limit_order = {
    "AccountKey": client_data["DefaultAccountKey"],
    "AssetType": eurusd_details["AssetType"],
    "Uic": eurusd_details["Uic"],
    "BuySell": "Buy",
    "Amount": 1000,
    "OrderType": "Stop",  # note: for exchange-traded instruments this field should be set to "StopIfTraded"
    "OrderPrice": str(order_price),  # for serialization
    "OrderDuration": {"DurationType": "GoodTillCancel"},
    "ExternalReference": token_urlsafe(16),  # a random identifier for this order
    "ManualOrder": True,
}

placed_order = client.post("/trade/v2/orders", data=limit_order)

print(f"Order placed with id: {placed_order['OrderId']}, external reference: {placed_order['ExternalReference']}")

# retrieve order status from portfolio
order = client.get(f"/port/v1/orders/{client_data['ClientKey']}/{placed_order['OrderId']}")["Data"][0]

print(f"Order: {order['OrderId']}, status: {order['Status']}")

# cancel order
client.delete(f"/trade/v2/orders/{order['OrderId']}?AccountKey={order['AccountKey']}")

# no data is returned because status of delete request is 204 No Content

# confirm order is now removed
client.get(f"/port/v1/orders/{client_data['ClientKey']}/{placed_order['OrderId']}")

Order price for this order: 1.12740
Order placed with id: 5010735122, external reference: O-m6hbs_802sSa9zu6sd4A
Order: 5010735122, status: Working


{'__count': 0, 'Data': []}